In [2]:
import pandas as pd
import pickle
import re
from pptx import Presentation

In [3]:
## Load pickle
with open("bible_dataframe.pickle","rb") as fr:
    bibleDF = pickle.load(fr)
with open("bible_dict.pickle","rb") as fr:
    bookDict = pickle.load(fr)

In [14]:
raw = """
                       【새 사람을 입으라(엡4:17-24절)】 

●2020년 12월 31일(송구영신예배)           ●301장(지금까지 지내온 것)

●17-18절.
“17 그러므로 내가 이것을 말하며 주 안에서 증언하노니, 이제부터는 너희는 이방인이 그 마음의 허망한 것으로 행함 같이 행하지 말라 / 18 그들의 총명이 어두워지고, 그들 가운데 있는 무지함과 그들의 마음이 굳어짐으로 말미암아, 하나님의 생명에서 떠나 있도다.” 

●19절.
“그들이 감각 없는 자가 되어, 자신을 방탕에 방임하여 모든 더러운 것을 욕심으로 행하되” 

●22절. 
“22 너희는 유혹의 욕심을 따라 썩어져 가는 구습을 따르는 옛 사람을 벗어 버리고” 

●23-24절. 
“23 오직 너희의 심령이 새롭게 되어 / 24 하나님을 따라 의와 진리의 거룩함으로 지으심을 받은, 새 사람을 입으라.”


●1절
“다른 세계관들은 우리를 삶의 기쁨 속에 앉혀 놓고 다가올 슬픔을 내다보게 하지만, 기독교는 신자들에게 이 세상의 슬픔 속에 앉아 장차 올 기쁨을 능히 맛보게 한다.”

●20-21절.
“20 오직 너희는 그리스도를 그같이 배우지 아니하였느니라 / 21 진리가 예수 안에 있는 것 같이, 너희가 참으로 그에게서 듣고 또한 그 안에서 가르침을 받았을진대” 

●고린도후서 5장 17절.
“그런즉 누구든지 그리스도 안에 있으면 새로운 피조물이라. 이전 것은 지나갔으니 보라 새 것이 되었도다.” 

●1절
“최대한 행복해 지는 것이, 모든 그리스도의 의무이다”

●1절
“하나님은 인간을 지으실 때도 다른 모든 피조물이 그런 것처럼 행복하도록 지으셨다. 행복한 것이 인간의 제 모습이다.”

●스바냐 3장 14절.
“시온의 딸아. 노래할지어다. 이스라엘아. 기쁘게 부를지어다. 예루살렘 딸아. 전심으로 기뻐하며(행복해하며) 즐거워할지어다.”
"""

In [15]:
# 성경 약자 반환 함수
def abbreviation(book):
    if book in list(bookDict.values()):
        book = list(bookDict.keys())[list(bookDict.values()).index(book)]
    return book

In [16]:
# 정규식 파싱 함수 
def parse_paragraph(paragraph):
    regex = re.compile('(?P<book>[ㄱ-ㅣ가-힣]+)\s*(?P<chapter>[0-9]+):?장?\s*(?P<verse_start>[0-9]+)-*(?P<verse_end>[0-9]+)*절?')
    parsed_paragraph = regex.search(paragraph)

    # print(parsed_paragraph.groups())

    book = parsed_paragraph.group('book')
    chapter = parsed_paragraph.group('chapter')
    verse_start = parsed_paragraph.group('verse_start')
    verse_end = parsed_paragraph.group('verse_end')

    if book in list(bookDict.values()):
        book = list(bookDict.keys())[list(bookDict.values()).index(book)]
        
    return book, chapter, verse_start, verse_end
# book, chapter, verse_start, verse_end = parse_paragraph(paragraph)

In [17]:
# 검색 구절을 사전형태로 반환하는 함수
def dict_contents(book, chapter, verse_start, verse_end):
    search_result = bibleDF.loc[ (bibleDF.book == book) & \
                                 (bibleDF.chapter == int(chapter)) & \
                                 (bibleDF.verse >= int(verse_start)) & \
                                 (bibleDF.verse <= int(verse_end)),
                                 'contents' ]
    verses = range(int(verse_start), int(verse_end)+1)
    keys = [ f"{bookDict[book]} {chapter}장 {i}절" for i in verses ]
    contentsDict = dict(zip(keys, search_result))
    return keys, contentsDict
# contentsDict = dict_contents(book, chapter, verse_start, verse_end)

In [18]:
def parsing_contents(raw):
    regex = re.compile(r'【\s*?(?P<title>.*?)\((?P<main_verse>.*?)\)?s*?】')
    main_title = regex.search(raw).group('title').strip()
    main_verse = regex.search(raw).group('main_verse')
    main_book, main_chapter, main_verse_start, main_verse_end = parse_paragraph(main_verse)
    
    regex = re.compile(r'●(.*?)\n“(.*?)”')
    all_p = regex.findall(raw)
    
    resultList = list()
    for i, c in all_p:
        regex = re.compile('(?P<book>[ㄱ-ㅣ가-힣]+)\s*(?P<chapter>[0-9]+):?장?\s*(?P<verse_start>[0-9]+)-*(?P<verse_end>[0-9]+)*절?')
        parsed_paragraph = regex.search(i)
        if parsed_paragraph:
            book = parsed_paragraph.group('book')
            chapter = parsed_paragraph.group('chapter')
            verse_start = parsed_paragraph.group('verse_start')
            verse_end = parsed_paragraph.group('verse_end')
            book = abbreviation(book)
        else:
            regex = re.compile('(?P<verse_start>[0-9]+)-*(?P<verse_end>[0-9]+)*절?')
            parsed_p = regex.search(i)
            book = main_book
            chapter = main_chapter
            verse_start = parsed_p.group('verse_start')
            verse_end = parsed_p.group('verse_end')
        if verse_end:
            contents = [ p.strip() for p in c.split('/') ]
        else:
            contents = [ c.strip() ]
        resultList.append((book, chapter, verse_start, verse_end, contents))
    return resultList, main_book, main_chapter, main_verse_start, main_verse_end, main_title

In [19]:
resultList, main_book, main_chapter, main_verse_start, main_verse_end, main_title = parsing_contents(raw)

In [20]:
 main_book, main_chapter, main_verse_start, main_verse_end, main_title

('엡', '4', '17', '24', '새 사람을 입으라')

In [21]:
keys, contentsDict = dict_contents(main_book, main_chapter, main_verse_start, main_verse_end)

In [22]:
from pptx import Presentation
from pptx.util import Pt

template_path='./verse_template.pptx'
save_file_name='new-file-name.pptx'


In [24]:
# def present(resultList=resultList, template_path=template_path, save_file_name=save_file_name, book=main_book, chapter=main_chapter, verse_start=main_verse_start, verse_end=main_verse_end, keys=keys, contentsDict=contentsDict, main_title=main_title):
prs = Presentation(template_path)

# 타이틀 삽입
title_slide_layout = prs.slide_layouts[0]
slide = prs.slides.add_slide(title_slide_layout)
title = slide.shapes.title
subtitle = slide.placeholders[1]

title.text = "성경봉독"
subtitle.text = f"{bookDict[main_book]} {main_chapter}장 {main_verse_start}-{main_verse_end}절"
# slide.shapes[2].text_frame.text = "(신약 p.)"

# 본문 삽입
for key in keys :
    contents_slide_layout = prs.slide_layouts[1]
    slide = prs.slides.add_slide(contents_slide_layout)
    title = slide.shapes.title
    contents = slide.placeholders[1]

    title.text = key
    contents.text = contentsDict[key]

#############        
# 타이틀 삽입
title_slide_layout = prs.slide_layouts[0]
slide = prs.slides.add_slide(title_slide_layout)
title = slide.shapes.title
subtitle = slide.placeholders[1]

title.text = "말씀선포"
subtitle.text = main_title

for book, chapter, verse_start, verse_end, contents in resultList:
    if not verse_end:
        key = f"{bookDict[book]} {chapter}장 {verse_start}절"
        # pptx
        contents_slide_layout = prs.slide_layouts[1]
        slide = prs.slides.add_slide(contents_slide_layout)
        title_placeholder = slide.shapes.title
        contents_placeholder = slide.placeholders[1]
        
        title_placeholder.text = key
        contents_placeholder.text = contents[0]
    else:
        key = f"{bookDict[book]} {chapter}장 {verse_start}-{verse_end}절" 
        
        contents_slide_layout = prs.slide_layouts[2]
        slide = prs.slides.add_slide(contents_slide_layout)
        
        title_placeholder = slide.shapes.title
        title_placeholder.text = key

        contents_text_frame = slide.placeholders[1].text_frame
        for _ in range(len(contents)-1):
            contents_text_frame.add_paragraph()
        for i in range(len(contents)):
            tmp1 = contents_text_frame.paragraphs[i].add_run()
            tmp1.text = re.search('\d+', contents[i]).group()
            tmp1.font.size = Pt(36)
            tmp2 = contents_text_frame.paragraphs[i].add_run()
            tmp2.text = re.sub('\d+', '', contents[i])

prs.save(save_file_name)

In [17]:
main_book, main_chapter, main_verse_start, main_verse_end = parse_paragraph("사도행전 3장 17-26절")
main_book, main_chapter, main_verse_start, main_verse_end

('행', '3', '17', '26')

In [18]:
keys, contentsDict = dict_contents(main_book, main_chapter, main_verse_start, main_verse_end)

In [22]:
from pptx import Presentation
from pptx.util import Pt

template_path='./verse_template.pptx'
save_file_name='new-file-name.pptx'

prs = Presentation(template_path)

# 본문 삽입
for key in keys :
    contents_slide_layout = prs.slide_layouts[1]
    slide = prs.slides.add_slide(contents_slide_layout)
    title = slide.shapes.title
    contents = slide.placeholders[1]

    title.text = key
    contents.text = contentsDict[key]

prs.save(save_file_name)

---

In [13]:
keys

['사도행전 2장 22절',
 '사도행전 2장 23절',
 '사도행전 2장 24절',
 '사도행전 2장 25절',
 '사도행전 2장 26절',
 '사도행전 2장 27절',
 '사도행전 2장 28절',
 '사도행전 2장 29절',
 '사도행전 2장 30절',
 '사도행전 2장 31절',
 '사도행전 2장 32절',
 '사도행전 2장 33절',
 '사도행전 2장 34절',
 '사도행전 2장 35절',
 '사도행전 2장 36절']

In [14]:
book, chapter, verse_start, verse_end, contents = resultList[0]
book, chapter, verse_start, verse_end, contents
key = f"{bookDict[book]} {chapter}장 {verse_start}-{verse_end}절" 
key

'사도행전 2장 22-24절'

In [15]:
[ book for book, chapter, verse_start, verse_end, contents in resultList ] 

['행', '행', '행', '막', '고전', '마', '마', '빌', '빌', '고후']

In [17]:
tmp1.group

AttributeError: '_Run' object has no attribute 'group'

In [28]:
contents

['그는 근본 하나님의 본체시나 하나님과 동등됨을 취할 것으로 여기지 아니하시고, 오히려 자기를 비워 종의 형체를 가지사 사람들과 같이 되셨고, 사람의 모양으로 나타나사 자기를 낮추시고 죽기까지 복종하셨으니 곧 십자가에 죽으심이라.']

In [29]:
resultList

[('행',
  '2',
  '22',
  '24',
  ['22 이스라엘 사람들아. 이 말을 들으라. 너희도 아는 바와 같이 하나님께서 나사렛 예수로 큰 권능과 기사와 표적을 너희 가운데서 베푸사, 너희 앞에서 그를 증언하셨느니라',
   '23 그가 하나님께서 정하신 뜻과 미리 아신대로 내준 바 되었거늘, 너희가 법 없는 자들의 손을 빌려 못 박아 죽였으나',
   '24 하나님께서 그를 사망의 고통에서 풀어 살리셨으니, 이는 그가 사망에 매여 있을 수 없었음이라.']),
 ('행',
  '2',
  '36',
  None,
  ['그런즉 이스라엘 온 집은 확실히 알지니, 너희가 십자가에 못 박은 이 예수를 하나님이 주와 그리스도가 되게 하셨느니라 하니라.']),
 ('행',
  '2',
  '29',
  '32',
  ['29 형제들아. 내가 조상 다윗에 대하여 담대히 말할 수 있노니, 다윗이 죽어 장사되어 그 묘가 오늘까지 우리 중에 있도다',
   '30 그는 선지자라. 하나님이 이미 맹세하사, 그 자손 중에서 한 사람을 그 위에 앉게 하리라 하심을 알고',
   '31 미리 본 고로 그리스도의 부활을 말하되, 그가 음부에 버림이 되지 않고, 그의 육신이 썩음을 당하지 아니하시리라 하더니',
   '32 이 예수를 하나님이 살리신지라. 우리가 다 이 일에 증인이로다.']),
 ('막',
  '1',
  '16',
  '20',
  ['16 갈리리 해변으로 지나가시다가, 시몬과 그 형제 안드레가 바다에 그물을 던지는 것을 보시니, 그들은 어부라',
   '17 예수께서 이르시되, 나를 따라오라. 내가 너희로 사람을 낚는 어부가 되게 하리라 하시니',
   '18 곧 그물을 버려 두고 따르니라',
   '19 조금 더 가시다가 세베대의 아들 야고보와 그 형제 요한을 보시니, 그들도 배에 있어 그물을 깁는데',
   '20 곧 부르시니, 그 아버지 세베대를 품꾼들과 함께 배에 버려 두고 예수를 따라가니라.']),
 ('고전',
  '9',
  '5',
  

In [10]:
text = """
3. 교육부서 임원, 교사 임명							
							
  - 대학청년부 (5명)							
"     (교역자) 이정오목사   (부장집사) 한병수  (회장) 천성택   (회계)유우종  (새가족반) 조현진
     "							
  - 중고등부 (4명)							
     (교역자) 정병호   (부장집사) 서두익 이현경  (총무, 교사) 박혜민  (회계) 유정란  							
  - 유초등부 (7명)							
     (교역자) 정병호   (부장집사) 김영희   (총무, 회계, 교사) 최승수 							
     (교사) 이동현  추진영  이예찬  (서기, 보조교사) 남성주							
  - 유치부 (7명)							
     (교역자) 조현진   (부장집사) 김덕수   (총무) 김혜림   (회계) 김준오							
     (교사) 김정미  천성택  최현경							

"""

In [11]:
text.strip().replace('\t','').replace('\n','')

'3. 교육부서 임원, 교사 임명  - 대학청년부 (5명)"     (교역자) 이정오목사   (부장집사) 한병수  (회장) 천성택   (회계)유우종  (새가족반) 조현진     "  - 중고등부 (4명)     (교역자) 정병호   (부장집사) 서두익 이현경  (총무, 교사) 박혜민  (회계) 유정란    - 유초등부 (7명)     (교역자) 정병호   (부장집사) 김영희   (총무, 회계, 교사) 최승수      (교사) 이동현  추진영  이예찬  (서기, 보조교사) 남성주  - 유치부 (7명)     (교역자) 조현진   (부장집사) 김덕수   (총무) 김혜림   (회계) 김준오     (교사) 김정미  천성택  최현경'

'- 일반재정부 (4명)\n     (부장) 윤영배  (위원) 오광수  이서영  이현재  \n  - 선교재정부 (2명)\n     (부장) 장현준  (위원) 김덕수  \n  - 교회이전위원회 (17명)\n     (위원장) 한병수  (총무) 오광수  (서기) 유우종  (회계 및 재정팀장) 장현준  (기술팀장) 서두익 \n     (행정팀장) 윤영배  (위원) 김덕수 김정도 서상욱 유안종 양하늬 이상필 이서영 이점달 이현재\n      장 원 최승수 \n  - 감사팀 (2명)\n    (팀장) 한병수  (팀원) 이점달\n  - 찬양팀(17명)\n    (팀장) 오광수  (총무) 이현경  (팀원) 강보배  김정미  김준오  김훈성  박아영  양진병  유안종\n     유우종  이다빈  임진슬  장원  정병호  천성택  최현경  허선혜"'